In [1]:
#!pip install lyricsgenius
#!pip install langdetect

In [2]:
import pandas as pd
import re
from lyricsgenius import Genius
from tqdm import tqdm
import sqlite3
from langdetect import detect
import spacy

nlp = spacy.load('en', disable=['parser', 'ner'])
conn = sqlite3.connect('songs.db')
c = conn.cursor()
genius = Genius('0NRl7k3ZzZXDaHFTkE882V0DP8nKbra_WoufZhtBjTngOjTxIa80_6u3W48RjIvt', skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], timeout=15, retries=3)

In [33]:
c.execute("CREATE TABLE Songs ('title', 'artists', 'featured artists', 'genre', 'lyrics')")

In [4]:
def get_genre_lyrics(genre):
    page = 1
    for _ in tqdm(range(50)):
        res = genius.tag(genre, page=page)
        for hit in res['hits']:
            song_lyrics = genius.lyrics(song_url=hit['url'])
            artists =  ' '.join(hit['artists'])
            featured_artists = ' '.join(hit['featured_artists'])
            c.execute('INSERT INTO Songs VALUES (?, ?, ?, ?, ?)', 
                      (hit['title'], artists, featured_artists,
                       genre, song_lyrics))
        page = res['next_page']

In [5]:
genres = ['pop', 'rap', 'rock', 'folk', 'indie']
for genre in genres:
    get_genre_lyrics(genre)
conn.commit()

100%|██████████| 50/50 [34:08<00:00, 40.97s/it]


In [3]:
from langdetect import detect
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

In [16]:
df = pd.read_sql('SELECT * FROM Songs', conn)
nonen = []
choruses = []
verses = []
songs = []
fulls = []
fullcs = []
fullvs = []
for n, song in tqdm(enumerate(df.lyrics)):
    lan = detect(song)
    if lan != 'en':
        nonen.append(n)
    else:
        chorus = ''
        verse = ''
        sg = ''
        full_c = []
        full_v = []
        part_types = re.findall('\[.+\]', song)
        parts = re.split('\[.+\]', song)
        full_s = '\n'.join(parts)
        for i, part_type in enumerate(part_types):
            doc = nlp(parts[i+1])
            part = ' '.join([token.lemma_ for token in doc])
            if '[Chorus' in part_type:
                chorus += part
                full_c += parts[i+1]
            else:
                verse += part
                full_v += parts[i+1]
            sg += part
        choruses.append(chorus)
        verses.append(verse)
        songs.append(sg)
        fulls.append(full_s)
        fullcs.append(''.join(full_c))
        fullvs.append(''.join(full_v))
        
df = df.drop(nonen)
df['clean_chorus'] = choruses
df['clean_verses'] = verses
df['clean_lyrics'] = songs
df['full_lyrics'] = fulls
df['full_chorus'] = fullcs
df['full_verses'] = fullvs

4999it [03:01, 27.57it/s]


In [17]:
df = df.drop(df[(df.clean_chorus == '') | (df.clean_verses == '')].index)

In [18]:
genres = ['pop', 'rap', 'rock', 'folk', 'indie']
for genre in genres:
    print(genre, len(df[df.genre==genre]))

pop 870
rap 777
rock 859
folk 634
indie 795


In [19]:
df.full_chorus.values[1]

"\nThank you, next (Next)\nThank you, next (Next)\nThank you, next\nI'm so fuckin' grateful for my ex\nThank you, next (Next)\nThank you, next (Next)\nThank you, next (Next)\nI'm so fuckin'—\n\nThank you, next (Thank you, next)\nThank you, next (Thank you, next)\nThank you, next (Thank you)\nI'm so fuckin' grateful for my ex\nThank you, next (Thank you, next)\nThank you, next (Said thank you, next)\nThank you, next (Next)\nI'm so fuckin' grateful for my ex\n\nThank you, next (Thank you, next)\nThank you, next (Thank you, next)\nThank you, next\nI'm so fuckin' grateful for my ex\nThank you, next (Thank you, next)\nThank you, next (Said thank you, next)\nThank you, next (Next)\nI'm so fuckin' grateful for my ex\n\n"

In [21]:
df.to_sql('Splitted', conn)
conn.commit()

In [22]:
df = pd.read_sql('SELECT * FROM Splitted', conn)

In [23]:
from spacy.lang.en.stop_words import STOP_WORDS

In [25]:
waste = ['-pron-', 'ya', 'e', 'hey', 'oh', 'la', 'uh', 'ayy', 'ooh', 'ah', 'da', 'mmm', 'ba', 'doo', 'woah', 'whoa', 'na', 'oo+h*', 'o', '-', 'm', 's', '\W+', 'hoh', 'yeah', 'yee']
def make_corpus(texts, filename):
    doc = ''
    for i, text in enumerate(texts):
        text = text.lower()
        song = re.sub('[^\w-]', ' ', text)
        song = re.sub(' +', ' ', song)
        clean_lr = []
        for word in song.split():
            lexeme = nlp.vocab[word]
            if lexeme.is_stop == False and word not in waste:
                clean_lr.append(word)
        song = ' '.join(clean_lr)
        doc += ('doc' + str(i) + ' ' + song.lower() + ' | genre ' + df.genre.values[i] + '\n')
    with open(filename, 'w') as f:
        f.write(doc)
    return doc, song

In [26]:
full_doc = make_corpus(df.clean_lyrics, 'full_corpus.txt')
ch_doc = make_corpus(df.clean_chorus, 'ch_corpus.txt')
v_doc = make_corpus(df.clean_lyrics, 'v_corpus.txt')